In [3]:
import requests
requests.get('https://dashscope.aliyuncs.com', verify=False)

/opt/anaconda3/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [404]>

In [ ]:
!pwd

In [5]:
import os
os.chdir('/Users/canglong/Documents/vscode_project/Learn-AI-Sample/6-RAG技术与应用/CASE1-ChatPDF-Faiss')
print(os.getcwd())  # 再次确认

/Users/canglong/Documents/vscode_project/Learn-AI-Sample/6-RAG技术与应用/CASE1-ChatPDF-Faiss


In [ ]:
!pip install langchain_openai langchain_community langchain PyPDF2 faiss-cpu

In [6]:
from PyPDF2 import PdfReader
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain_community.callbacks.manager import get_openai_callback
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from typing import List, Tuple, Dict

# Qwen-Turbo做推理模型
DASHSCOPE_API_KEY = 'sk-xxxx'

def extract_text_with_pages(pdf) -> Dict[int, str]:
    """
    从PDF中提取文本，按页面分别存储
    
    参数:
        pdf: PDF文件对象
    
    返回:
        pages_text: 字典，键为页码，值为该页的文本内容
    """
    pages_text = {}
    
    for page_number, page in enumerate(pdf.pages, start=1):
        extracted_text = page.extract_text()
        if extracted_text:
            pages_text[page_number] = extracted_text.strip()
        else:
            pages_text[page_number] = ""
            print(f"Warning: No text found on page {page_number}.")
    
    return pages_text

# 读取PDF文件
pdf_reader = PdfReader('./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf')
# 提取每页文本
pages_text = extract_text_with_pages(pdf_reader)

# 合并所有页面文本
full_text = "\n\n".join([f"[页面 {page_num}]\n{content}" for page_num, content in pages_text.items()])

print(f"PDF共有 {len(pages_text)} 页")
for page_num, content in pages_text.items():
    print(f"第 {page_num} 页文本长度: {len(content)} 字符")

full_text[:500]  # 显示前500个字符

PDF共有 9 页
第 1 页文本长度: 438 字符
第 2 页文本长度: 381 字符
第 3 页文本长度: 434 字符
第 4 页文本长度: 420 字符
第 5 页文本长度: 537 字符
第 6 页文本长度: 511 字符
第 7 页文本长度: 567 字符
第 8 页文本长度: 474 字符
第 9 页文本长度: 93 字符


'[页面 1]\n百度文库  - 好好学习，天天向上  \n-1 上海浦东发展银行西安分行  \n个金客户经理管理考核暂行办法  \n \n \n第一章  总   则 \n第一条   为保证我分行个金客户经理制的顺利实施，有效调动个\n金客户经理的积极性，促进个金业务快速、稳定地发展，根据总行《上\n海浦东发展银行个人金融营销体系建设方案（试行）》要求，特制定\n《上海浦东发展银行西安分行个金客户经理管理考核暂行办法（试\n行）》（以下简称本办法）。  \n第二条   个金客户经理系指各支行（营业部）从事个人金融产品\n营销与市场开拓，为我行个人客户提供综合银行服务的我行市场人\n员。 \n第三条   考核内容分为二大类， 即个人业绩考核、 工作质量考核。\n个人业绩包括个人资产业务、负债业务、卡业务。工作质量指个人业\n务的资产质量。  \n第四条   为规范激励规则，客户经理的技术职务和薪资实行每年\n考核浮动。客户经理的奖金实行每季度考核浮动，即客户经理按其考\n核内容得分与行员等级结合，享受对应的行员等级待遇。\n\n[页面 2]\n百度文库  - 好好学习，天天向上  \n-2 第二章  职位设置与职责  \n第五条   个金'

In [7]:
def process_text_with_accurate_page_mapping(pages_text: Dict[int, str]) -> FAISS:
    """
    处理文本并创建带有精确页码映射的向量存储
    
    参数:
        pages_text: 每页的文本内容字典
    
    返回:
        knowledgeBase: 基于FAISS的向量存储对象，包含页码信息
    """
    # 创建文本分割器
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " ", ""],
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    
    all_chunks = []
    chunk_to_page = {}  # 存储每个chunk对应的页码
    
    # 对每页文本分别进行分割
    for page_num, page_text in pages_text.items():
        if page_text.strip():  # 只处理非空页面
            # 为该页文本添加页面标识
            marked_text = f"[页面 {page_num}]\n{page_text}"
            
            # 分割该页文本
            page_chunks = text_splitter.split_text(marked_text)
            
            # 记录每个chunk对应的页码
            for chunk in page_chunks:
                all_chunks.append(chunk)
                chunk_to_page[chunk] = page_num
    
    print(f"总共创建了 {len(all_chunks)} 个文本块")
    
    # 创建嵌入模型
    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1",
        dashscope_api_key=DASHSCOPE_API_KEY,
    )
    
    # 从文本块创建知识库
    knowledgeBase = FAISS.from_texts(all_chunks, embeddings)
    
    # 将页码映射信息存储到知识库对象中
    knowledgeBase.chunk_to_page = chunk_to_page
    
    # 调试信息：显示每个chunk对应的页码
    for i, chunk in enumerate(all_chunks):
        page = chunk_to_page.get(chunk, "未知")
        print(f"块 {i+1} 对应页码: {page}")
        print(f"块内容预览: {chunk[:100]}...")
        print("-" * 50)
    
    return knowledgeBase

# 处理文本并创建知识库
knowledgeBase = process_text_with_accurate_page_mapping(pages_text)

总共创建了 9 个文本块
块 1 对应页码: 1
块内容预览: [页面 1]
百度文库  - 好好学习，天天向上  
-1 上海浦东发展银行西安分行  
个金客户经理管理考核暂行办法  
 
 
第一章  总   则 
第一条   为保证我分行个金客户经理制的顺利...
--------------------------------------------------
块 2 对应页码: 2
块内容预览: [页面 2]
百度文库  - 好好学习，天天向上  
-2 第二章  职位设置与职责  
第五条   个金客户经理职位设置为：客户经理助理、客户经理、
高级客户经理、资深客户经理。  
第六条   个...
--------------------------------------------------
块 3 对应页码: 3
块内容预览: [页面 3]
百度文库  - 好好学习，天天向上  
-3 第三章  基础素质要求  
第七条   个金客户经理准入条件：  
（一）工作经历：须具备大专以上学历，至少二年以上银行工作
经验。  
（...
--------------------------------------------------
块 4 对应页码: 4
块内容预览: [页面 4]
百度文库  - 好好学习，天天向上  
-4 3 100  
2 105  
1 110  
客户经理 5 115 300万  500张 
4 120  
3 125  
2 130  ...
--------------------------------------------------
块 5 对应页码: 5
块内容预览: [页面 5]
百度文库  - 好好学习，天天向上  
-5 第五章  工作质量考核标准  
第九条   工作质量考核实行扣分制。工作质量指个金客户经理在
从事所有个人业务时出现投诉、差错及风险。该项考...
--------------------------------------------------
块 6 对应页码: 6
块内容预览: [页面 6]
百度文库  - 好好学习，天天向上  
-6 C.发生逾期超过 3个月，无论金额大小和笔数，扣 10分。 
 
第六章  聘任考核程序  
第十条   凡达到本办法第三章规定的该技术

In [9]:
# 保存FAISS向量数据库
knowledgeBase.save_local('./faiss-1')

In [10]:
from langchain_community.llms import Tongyi
llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY) # qwen-turbo

# 设置查询问题
query = "客户经理被投诉了，投诉一次扣多少分"
# query = "客户经理每年评聘申报时间是怎样的？"
if query:
    # 执行相似度搜索，找到与查询相关的文档
    docs = knowledgeBase.similarity_search(query,k=2)

    # 加载问答链
    chain = load_qa_chain(llm, chain_type="stuff")

    # 准备输入数据
    input_data = {"input_documents": docs, "question": query}

    # 使用回调函数跟踪API调用成本
    with get_openai_callback() as cost:
        # 执行问答链
        response = chain.invoke(input=input_data)
        print(f"查询已处理。成本: {cost}")
        print(response["output_text"])
        print("来源:")

    # 显示每个文档块的来源页码
    for i, doc in enumerate(docs):
        text_content = getattr(doc, "page_content", "")
        source_page = knowledgeBase.chunk_to_page.get(text_content, "未知")
        print(f"文本块 {i+1} 页码: {source_page}")
        print(f"内容预览: {text_content}...")
        print("-" * 30)

/var/folders/v5/8q690d8n5ql_dmytmb8mgzxh0000gp/T/ipykernel_8297/1460356726.py:12: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


查询已处理。成本: Tokens Used: 0
	Prompt Tokens: 0
		Prompt Tokens Cached: 0
	Completion Tokens: 0
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0
根据提供的信息，客户经理如果被投诉一次，扣 **2分**。
来源:
文本块 1 页码: 5
内容预览: [页面 5]
百度文库  - 好好学习，天天向上  
-5 第五章  工作质量考核标准  
第九条   工作质量考核实行扣分制。工作质量指个金客户经理在
从事所有个人业务时出现投诉、差错及风险。该项考核最多扣 50分，
如发生重大差错事故，按分行有关制度处理。  
（一）服务质量考核：   
1、工作责任心不强，缺乏配合协作精神；扣 5分 
2、客户服务效率低，态度生硬或不及时为客户提供维护服务，
有客户投诉的 ,每投诉一次扣 2分 
3、不服从支行工作安排，不认真参加分（支）行宣传活动的，
每次扣 2分； 
4、未能及时参加分行（支行）组织的各种业务培训、考试和专
题活动的每次扣 2分； 
5、未按规定要求进行贷前调查、贷后检查工作的，每笔扣 5分； 
6、未建立信贷台帐资料及档案的每笔扣 5分； 
7、在工作中有不廉洁自律情况的每发现一次扣 50分。 
（二）个人资产质量考核：  
当季考核收息率 97%以上为合格，每降 1个百分点扣 2分；不
良资产零为合格，每超一个个百分点扣 1分。 
A.发生跨月逾期，单笔不超过 10万元，当季收回者，扣 1分。 
B.发生跨月逾期， 2笔以上累计金额不超过 20万元，当季收回
者，扣 2分；累计超过 20万元以上的，扣 4分。...
------------------------------
文本块 2 页码: 8
内容预览: [页面 8]
百度文库  - 好好学习，天天向上  
-8 第十五条   各项考核分值总计达到某一档行员级别考核分值标
准，个金客户经理即可在下一季度享受该级行员的薪资标准。下一季
度考核时，按照已享受行员级别考核折算比值进行考核，以次类推。  
第十六条   对已聘为各级客户经理的人员，当工作业绩考核达不
到相应技术职务要求下限时，下一年技术职务相应下调 。 
第十七条   为保

In [ ]:
# 测试修复后的页码映射
print("=== 测试页码映射修复 ===")

# 重新运行整个流程
pdf_reader = PdfReader('./浦发上海浦东发展银行西安分行个金客户经理考核办法.pdf')
pages_text = extract_text_with_pages(pdf_reader)
knowledgeBase = process_text_with_accurate_page_mapping(pages_text)

# 测试查询
query = "客户经理被投诉了，投诉一次扣多少分"
docs = knowledgeBase.similarity_search(query, k=4)

print(f"\n查询: {query}")
print(f"返回了 {len(docs)} 个相关文档块")

for i, doc in enumerate(docs):
    text_content = getattr(doc, "page_content", "")
    source_page = knowledgeBase.chunk_to_page.get(text_content, "未知")
    print(f"\n文档块 {i+1}:")
    print(f"页码: {source_page}")
    print(f"内容: {text_content[:300]}...")
    print("-" * 50)